# Test bug on RAGxplorer on query
Without brackets around the query in the openai client with chroma, embeddings are created for each word, not each string.

In [2]:
import os
from ragxplorer import RAGxplorer

from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)
import json
import chromadb
import pandas as pd

KeyboardInterrupt: 

In [ ]:
query="What are examples of lubricants which should be avoided for space mechanism applications?"

OPENAI_API_KEY=os.getenv('OPENAI_API_KEY')
embedding_model='text-embedding-ada-002'

data_path='../data/AMS/'
pdf='AMS_2022.pdf'

index_path='../db/chromadb/'
index_name='chromadb-openai-ams'

viz_data='../data/AMS/ams_data-400-0-50.json'

In [ ]:
with open(viz_data, 'r') as f:
    data = json.load(f)

index_name=data['visualization_index_name']
umap_params=data['umap_params']
viz_data=pd.read_json(data['viz_data'], orient='split')

In [ ]:
rx_client = RAGxplorer(embedding_model=embedding_model)
chroma_client = chromadb.PersistentClient(path='../db'+'/chromadb/')
collection=chroma_client.get_collection(name=index_name,embedding_function=rx_client._chosen_embedding_model)

In [ ]:
rx_client._query.original_query = query

In [ ]:
rx_client.load_chroma(collection,
                     umap_params=umap_params,
                     initialize_projector=True)

In [ ]:
# All of these do the same thing, tested

query_embedding_chroma=rx_client._chosen_embedding_model([rx_client._query.original_query])
# query_embedding=rx_client._vectordb._embedding_function(rx_client._query.original_query)
# query_embedding=embedding_function(query)

In [ ]:
len(rx_client._query.original_query)

In [ ]:
fig = rx_client.visualize_query(query,
                                import_projection_data=viz_data,
                                verbose=True)

In [ ]:
from openai import OpenAI
client = OpenAI()

# Done correctly, see here: https://platform.openai.com/docs/guides/embeddings/use-cases
openai_embedding=client.embeddings.create(input = [query], model=embedding_model).data[0].embedding

# Endpoints

In [ ]:
import json

config_file = '../config/config.json'
with open(config_file, 'r') as f:
        config = json.load(f)
        databases = {db['name']: db for db in config['databases']}
        llms  = {m['name']: m for m in config['llms']}

In [ ]:
hf_models=llms['Hugging Face']['models']
model_names= [item['model'] for item in hf_models]
model_endpoints= [item['endpoint'] for item in hf_models]


In [ ]:
model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
endpoint = next(item['endpoint'] for item in hf_models if item['model'] == model_name)

In [ ]:
from langchain_openai import OpenAI, ChatOpenAI
# from langchain_community.llms import HuggingFaceTextGenInference
from langchain_community.llms import HuggingFaceHub
import os

from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)

api_key=os.getenv('HUGGINGFACEHUB_API_TOKEN')

# llm = ChatOpenAI(
#                     model_name="tgi",
#                     openai_api_key=api_key,
#                     openai_api_base=endpoint + "/v1/",
#                 )




# llm = HuggingFaceTextGenInference(
#     inference_server_url=endpoint,
#     # max_new_tokens=512,
#     # top_k=10,
#     # top_p=0.95,
#     # typical_p=0.95,
#     temperature=0.01,
#     # repetition_penalty=1.03,
# )
# llm("What did foo say about bar?")

# model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
# model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
# model_name = "meta-llama/Llama-2-70b-chat-hf"

llm = HuggingFaceHub(repo_id=model_name,
                     model_kwargs={"temperature": 0.1, "max_length": 516})


In [ ]:
prompt="""
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
---
Your name is Aerospace Chatbot. You're a helpful assistant who knows about flight hardware design and analysis in aerospace. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Include sources from the chat history in the standalone question created.
---

Chat History:

User Question: What are examples of lubricants which should be avoided for space mechanism applications?
Standalone Question:
"""

response=llm.predict(prompt)
print(response)

# Advanced RAG

In [59]:
import os
import chromadb

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document as lancghain_Document
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Pinecone

from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel
from langchain.prompts.prompt import PromptTemplate

from pinecone import Pinecone as pinecone_client

from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)

local_db_path='../db/'
chunk_size=500
chunk_overlap=0
embedding_model='text-embedding-ada-002'

llm = ChatOpenAI(openai_api_key=os.getenv('OPENAI_API_KEY'), model_name="gpt-3.5-turbo", temperature=0.1)
query_model=OpenAIEmbeddings(model=embedding_model,openai_api_key=os.getenv('OPENAI_API_KEY'))
store=InMemoryStore()


In [60]:
index_type='chroma'
# index_type='pinecone'

type='standard'
# type='parent-child'

if index_type=='chroma':
    # Chroma
    persistent_client = chromadb.PersistentClient(path=local_db_path+'/chromadb')   
    vectorstore = Chroma(client=persistent_client,
                        collection_name='parent-child-test',
                        embedding_function=query_model)
elif index_type=='pinecone':
    # Pinecone
    pc=pinecone_client(api_key=os.getenv('PINECONE_API_KEY'))
    if type=='standard':
        index=pc.Index('pinecone-openai-ams')
    elif type=='parent-child':
        index=pc.Index('pinecone-openai-ams-pc')
    vectorstore = Pinecone(index, query_model, "page_content")

In [57]:
# persistent_client.delete_collection(name="parent-child-test")

In [61]:
docs=['../data/AMS/AMS_2020.pdf','../data/AMS/AMS_2018.pdf']

In [62]:
docs_out=[]
for doc in docs:
    loader = PyPDFLoader(doc)
    data = loader.load()
    docs_out.extend(data)

In [63]:
data[101]

Document(page_content='Motivation and Background \nBall bearing systems in space mechanisms are usually composed of two angular contact or deep-groove \nbearings, which are preloaded along their axial direction in order to ensure a sufficiently high stiffness, the \nrequired precision of rotation, and a stable friction torque (within acceptable limits). The bearing preload \ncan be applied using two different basic methods: hard or soft preload. In the hard preload method, a \ndeflection is forced on the bearing pair by rigid mounting parts, corresponding to the desired preload \nmagnitude (knowing the load-deflection characteristic of the bearings). The soft preload method relies on a \ncompliant element, typically some spring-type component, to apply an axial force on the bearing pair \ncorresponding to the controlled deflection of the compliant element (i.e., knowing its load-deflection curve). \nBoth methods have their pros and cons [3] that will not be further detailed here for br

In [64]:
questions=["What are some challenges associated with light cover mechanisms?"]

In [65]:
from langchain import hub
from langchain.prompts.prompt import PromptTemplate

# Prompts on the hub: https://smith.langchain.com/hub/my-prompts?organizationId=45eb8917-7353-4296-978d-bb461fc45c65
CONDENSE_QUESTION_PROMPT = hub.pull("dmueller/ams-chatbot-qa-condense-history")
QA_PROMPT=hub.pull("dmueller/ams-chatbot-qa-retrieval")
QA_WSOURCES_PROMPT=hub.pull("dmueller/ams-chatbot-qa-retrieval-wsources")
QA_GENERATE_PROMPT=hub.pull("dmueller/generate_qa_prompt")
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")

In [66]:
# Combine documents, from queries.py

def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [67]:
# From queries.py

memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

## Standard RAG

In [68]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
page_chunks = text_splitter.split_documents(docs_out)
vectorstore.add_documents(page_chunks)

['27e41c30-cdda-11ee-ab51-2e02fffc2e16',
 '27e41d7a-cdda-11ee-ab51-2e02fffc2e16',
 '27e41d98-cdda-11ee-ab51-2e02fffc2e16',
 '27e41da2-cdda-11ee-ab51-2e02fffc2e16',
 '27e41db6-cdda-11ee-ab51-2e02fffc2e16',
 '27e41dc0-cdda-11ee-ab51-2e02fffc2e16',
 '27e41dca-cdda-11ee-ab51-2e02fffc2e16',
 '27e41dde-cdda-11ee-ab51-2e02fffc2e16',
 '27e41de8-cdda-11ee-ab51-2e02fffc2e16',
 '27e41dfc-cdda-11ee-ab51-2e02fffc2e16',
 '27e41e06-cdda-11ee-ab51-2e02fffc2e16',
 '27e41e10-cdda-11ee-ab51-2e02fffc2e16',
 '27e41e24-cdda-11ee-ab51-2e02fffc2e16',
 '27e41e2e-cdda-11ee-ab51-2e02fffc2e16',
 '27e41e38-cdda-11ee-ab51-2e02fffc2e16',
 '27e41e4c-cdda-11ee-ab51-2e02fffc2e16',
 '27e41e56-cdda-11ee-ab51-2e02fffc2e16',
 '27e41e60-cdda-11ee-ab51-2e02fffc2e16',
 '27e41e6a-cdda-11ee-ab51-2e02fffc2e16',
 '27e41e7e-cdda-11ee-ab51-2e02fffc2e16',
 '27e41e88-cdda-11ee-ab51-2e02fffc2e16',
 '27e41e92-cdda-11ee-ab51-2e02fffc2e16',
 '27e41ea6-cdda-11ee-ab51-2e02fffc2e16',
 '27e41eb0-cdda-11ee-ab51-2e02fffc2e16',
 '27e41ec4-cdda-

In [69]:
retriever=vectorstore.as_retriever()

In [85]:
len(list(store.yield_keys()))

1150

In [70]:
# Directly from queries.py
# This adds a 'memory' key to the input object
loaded_memory = RunnablePassthrough.assign(
                    chat_history=RunnableLambda(memory.load_memory_variables) 
                    | itemgetter('history'))  

# Assemble main chain
standalone_question = {
    'standalone_question': {
        'question': lambda x: x['question'],
        'chat_history': lambda x: get_buffer_string(x['chat_history'])}
    | CONDENSE_QUESTION_PROMPT
    | llm
    | StrOutputParser()}

retrieved_documents = {
    'source_documents': itemgetter('standalone_question') 
                        | retriever,
    'question': lambda x: x['standalone_question']}

# Now we construct the inputs for the final prompt
final_inputs = {
    'context': lambda x: _combine_documents(x['source_documents']),
    'question': itemgetter('question')}

# And finally, we do the part that returns the answers
answer = {
    'answer': final_inputs 
                | QA_PROMPT 
                | llm,
    'references': itemgetter('source_documents')}
conversational_qa_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [71]:
inputs = {"question": questions[0]}
result = conversational_qa_chain.invoke(inputs)
result

{'answer': AIMessage(content="I'm sorry, I don't have information on a specific light cover mechanism that has faced challenges in its implementation."),
 'references': [Document(page_content='69 schedule. Furthermore, these solutions may appear straightforward, now that the development is complete, \nbut they were certainly not as obvious at the beginning of the project. That is precisely why it is important \nto look back to t he now completed development and identify important lessons that would help future \ndevelopment s.  \n \nThese lessons learned presented here are two- fold. There are specific lessons about what could be done \nfrom the beginning if a very similar mechanism were to  be developed. Then there are more generic lessons \nabout aspects which had an important impact on the CLUPI Focus Mechanism but are also applicable to a \nlot of  other  mechanism developments .  \n \nImprovement for a F uture Focus Mechanism  \nSolutions were found to answer the issues and addres

In [72]:
len(result['references'][0].page_content)

1822

## Parent-child
https://colab.research.google.com/github/datastax/ragstack-ai/blob/main/examples/notebooks/advancedRAG.ipynb

In [89]:
persistent_client.delete_collection(name="parent-child-test-pc")

In [90]:
index_type='chroma'
# index_type='pinecone'

# type='standard'
type='parent-child'

if index_type=='chroma':
    # Chroma
    persistent_client = chromadb.PersistentClient(path=local_db_path+'/chromadb')   
    vectorstore = Chroma(client=persistent_client,
                        collection_name='parent-child-test-pc',
                        embedding_function=query_model)
elif index_type=='pinecone':
    # Pinecone
    pc=pinecone_client(api_key=os.getenv('PINECONE_API_KEY'))
    if type=='standard':
        index=pc.Index('pinecone-openai-ams')
    elif type=='parent-child':
        index=pc.Index('pinecone-openai-ams-pc')
    vectorstore = Pinecone(index, query_model, "page_content")
store_parent_child=InMemoryStore()

In [91]:
k_parent=5
parent_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size*k_parent, chunk_overlap=chunk_overlap)
child_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

# Create a parent document retriever
parent_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store_parent_child,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [92]:
# Split and load the documents into the vector and parent stores
parent_retriever.add_documents(docs_out)

In [93]:
len(list(store.yield_keys()))

1150

In [80]:
# From queries.py

memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [81]:
retriever=parent_retriever

# Directly from queries.py
# This adds a 'memory' key to the input object
loaded_memory = RunnablePassthrough.assign(
                    chat_history=RunnableLambda(memory.load_memory_variables) 
                    | itemgetter('history'))  

# Assemble main chain
standalone_question = {
    'standalone_question': {
        'question': lambda x: x['question'],
        'chat_history': lambda x: get_buffer_string(x['chat_history'])}
    | CONDENSE_QUESTION_PROMPT
    | llm
    | StrOutputParser()}

retrieved_documents = {
    'source_documents': itemgetter('standalone_question') 
                        | retriever,
    'question': lambda x: x['standalone_question']}

# Now we construct the inputs for the final prompt
final_inputs = {
    'context': lambda x: _combine_documents(x['source_documents']),
    'question': itemgetter('question')}

# And finally, we do the part that returns the answers
answer = {
    'answer': final_inputs 
                | QA_PROMPT 
                | llm,
    'references': itemgetter('source_documents')}
conversational_qa_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [82]:
inputs = {"question": questions[0]}
result = conversational_qa_chain.invoke(inputs)
result

{'answer': AIMessage(content="I'm sorry, I don't have information on specific challenges that arise from light cover mechanisms."),
 'references': [Document(page_content='69 schedule. Furthermore, these solutions may appear straightforward, now that the development is complete, \nbut they were certainly not as obvious at the beginning of the project. That is precisely why it is important \nto look back to t he now completed development and identify important lessons that would help future \ndevelopment s.  \n \nThese lessons learned presented here are two- fold. There are specific lessons about what could be done \nfrom the beginning if a very similar mechanism were to  be developed. Then there are more generic lessons \nabout aspects which had an important impact on the CLUPI Focus Mechanism but are also applicable to a \nlot of  other  mechanism developments .  \n \nImprovement for a F uture Focus Mechanism  \nSolutions were found to answer the issues and address the challenges of th

In [83]:
len(result['references'][0].page_content)

1822

# Module testing

In [1]:
import sys
# Assuming `/home/user/projects/my_modules` is the path to the folder with your modules
sys.path.append('../src')

In [4]:
import data_processing
import glob

In [ ]:
data_folder = '../data/AMS/'
docs = glob.glob(data_folder+'*.pdf')   # Only get the PDFs in the directory

json_file = data_folder+'ams_data-400-0-0.json'
chunk_size = 500
chunk_overlap = 0
use_json = False
show_progress = True

data_processing.chunk_docs(docs,
                        file=json_file,
                        chunk_size=chunk_size,
                        chunk_overlap=chunk_overlap,
                        use_json=use_json,
                        show_progress=show_progress)